<a href="https://colab.research.google.com/github/Elwing-Chou/ml1216/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from urllib.request import urlretrieve
url = "https://github.com/Elwing-Chou/ml1216/raw/main/train.csv"
urlretrieve(url, "train.csv")
url = "https://github.com/Elwing-Chou/ml1216/raw/main/test.csv"
urlretrieve(url, "test.csv")

In [ ]:
import pandas as pd
train = pd.read_csv("train.csv", encoding="utf-8")
test = pd.read_csv("test.csv", encoding="utf-8")

In [ ]:
datas = pd.concat([train, test], axis=0, ignore_index=True)
datas = datas.drop(["PassengerId", "Survived"], axis=1)
datas

In [ ]:
s = datas.isna().sum()
# series[跟資料筆數依樣多的true/false]
s[s > 0].sort_values(ascending=False)

In [ ]:
count = datas["Ticket"].value_counts() 
def share(tn):
    return count[tn]
datas["Ticket"] = datas["Ticket"].apply(share)

In [ ]:
med = datas.median().drop("Pclass")
med

In [ ]:
datas = datas.fillna(med)

In [ ]:
most = datas["Embarked"].value_counts().idxmax()
datas["Embarked"] = datas["Embarked"].fillna(most)

In [ ]:
def cabin(c):
    if pd.isna(c):
        return None
    else:
        return c[0]
datas["Cabin"] = datas["Cabin"].apply(cabin)

In [ ]:
def name(n):
    n = n.split(",")[-1].split(".")[0]
    return n.strip()
count = datas["Name"].apply(name).value_counts()
# ['Mr', 'Miss', 'Mrs', 'Master']
reserved = count[:4].index
def name(n):
    n = n.split(",")[-1].split(".")[0]
    n = n.strip()
    if n in reserved:
        return n
    else:
        return None
datas["Name"] = datas["Name"].apply(name)

In [ ]:
datas

In [ ]:
datas = pd.get_dummies(datas)
datas = pd.get_dummies(datas, columns=["Pclass"])
datas

In [ ]:
datas["Family"] = datas["SibSp"] + datas["Parch"]
datas

In [ ]:
# iloc: [1st, 2nd, ....]
x_train = datas.iloc[:len(train)]
y_train = train["Survived"]
x_test = datas.iloc[len(train):]
x_test

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
params = {
   "n_estimators":range(20, 200, 5),
   "max_depth":range(5, 10)
}
clf = RandomForestClassifier()
search = GridSearchCV(clf, params, n_jobs=-1, cv=10)
search.fit(x_train, y_train)
print(search.best_score_)
print(search.best_params_)

In [ ]:
import numpy as np
from sklearn.model_selection import cross_val_score
clf = RandomForestClassifier(n_estimators=25, max_depth=5)
scores = cross_val_score(clf, x_train, y_train, n_jobs=-1, cv=10)
print(scores)
print(np.average(scores))